# Custom Workflows with Neural Heatmap API

This notebook demonstrates how to combine multiple API operations into custom workflows.

## What You'll Learn
- Combine multiple API calls into workflows
- Create custom analysis pipelines
- Automate data collection and export
- Build real-time monitoring systems

In [ ]:
# Import required libraries
import asyncio
from datetime import datetime
from pathlib import Path
import json

from neural_heatmap import (
    NeuralHeatmapClient, 
    connect, 
    FilterConfig, 
    VisualizationTheme,
    ExportFormat
)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries loaded successfully!")

## 1. Workflow: Comprehensive Analysis Pipeline

Combine correlation, temporal, and anomaly analysis into a single workflow.

In [ ]:
async def comprehensive_analysis_workflow(client, output_dir="./analysis_results"):
    """
    Run a comprehensive analysis combining multiple API operations.
    
    This workflow:
    1. Gets correlation matrix
    2. Analyzes temporal patterns
    3. Detects anomalies
    4. Exports all results
    """
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    print("🔍 Starting Comprehensive Analysis Workflow")
    print("=" * 50)
    
    # 1. Get correlation matrix
    print("\n1️⃣ Fetching correlation matrix...")
    correlation_df = await client.get_correlation_matrix(as_dataframe=True)
    print(f"   ✓ Retrieved {correlation_df.shape[0]}x{correlation_df.shape[1]} matrix")
    
    # 2. Get temporal patterns
    print("\n2️⃣ Analyzing temporal patterns...")
    temporal_patterns = await client.get_temporal_patterns()
    print(f"   ✓ Found {len(temporal_patterns)} patterns")
    
    # 3. Get anomalies
    print("\n3️⃣ Detecting anomalies...")
    anomalies = await client.get_anomalies()
    print(f"   ✓ Found {len(anomalies)} anomalies")
    
    # 4. Get temporal statistics
    print("\n4️⃣ Gathering temporal statistics...")
    temporal_stats = await client.get_temporal_statistics()
    print(f"   ✓ Statistics collected")
    
    # 5. Export all data
    print("\n5️⃣ Exporting results...")
    correlation_df.to_csv(output_path / "correlation_matrix.csv")
    print(f"   ✓ Exported correlation matrix")
    
    # Export patterns and anomalies as JSON
    results = {
        "timestamp": datetime.now().isoformat(),
        "correlation": {
            "shape": correlation_df.shape,
            "layers": list(correlation_df.columns)
        },
        "temporal_patterns": [
            {
                "type": p.pattern_type,
                "confidence": p.confidence,
                "frequency": p.frequency,
                "metadata": p.metadata
            }
            for p in temporal_patterns
        ],
        "anomalies": [
            {
                "type": a.anomaly_type,
                "severity": a.severity,
                "score": a.score,
                "description": a.description
            }
            for a in anomalies
        ],
        "temporal_stats": temporal_stats
    }
    
    with open(output_path / "comprehensive_analysis.json", "w") as f:
        json.dump(results, f, indent=2)
    print(f"   ✓ Exported analysis results")
    
    print("\n" + "=" * 50)
    print("✅ Comprehensive Analysis Complete!")
    print(f"   Results saved to: {output_path.absolute()}")
    
    return results

# Connect and run workflow
client = await connect("http://localhost:8080")
results = await comprehensive_analysis_workflow(client)

## 2. Workflow: Filtered Multi-Model Comparison

Compare correlations across multiple models with custom filters.

In [ ]:
async def multi_model_comparison_workflow(client, model_ids, layer_filter=None):
    """
    Compare correlations across multiple models with optional layer filtering.
    """
    print(f"🔬 Multi-Model Comparison Workflow")
    print("=" * 50)
    print(f"Models: {model_ids}")
    if layer_filter:
        print(f"Layer Filter: {layer_filter}")
    print()
    
    comparison_results = {}
    
    for model_id in model_ids:
        print(f"\n📊 Analyzing model: {model_id}")
        
        # Apply filter if specified
        if layer_filter:
            await client.set_filter(
                model_ids=[model_id],
                layer_ids=layer_filter
            )
        else:
            await client.set_filter(model_ids=[model_id])
        
        # Get correlation matrix
        try:
            corr_df = await client.get_correlation_matrix(as_dataframe=True)
            comparison_results[model_id] = {
                "correlation_matrix": corr_df,
                "mean_correlation": corr_df.values[np.triu_indices_from(corr_df.values, k=1)].mean(),
                "max_correlation": corr_df.values[np.triu_indices_from(corr_df.values, k=1)].max(),
                "min_correlation": corr_df.values[np.triu_indices_from(corr_df.values, k=1)].min()
            }
            print(f"  Mean Correlation: {comparison_results[model_id]['mean_correlation']:.3f}")
        except Exception as e:
            print(f"  Error: {e}")
            comparison_results[model_id] = None
    
    # Clear filter
    await client.clear_filter()
    
    # Create comparison summary
    print("\n" + "=" * 50)
    print("📈 Comparison Summary")
    print(f"{'Model':<20} {'Mean':<10} {'Max':<10} {'Min':<10}")
    print("-" * 50)
    
    for model_id, results in comparison_results.items():
        if results:
            print(f"{model_id:<20} {results['mean_correlation']:<10.3f} "
                  f"{results['max_correlation']:<10.3f} {results['min_correlation']:<10.3f}")
    
    return comparison_results

# Example: Compare multiple models
# comparison = await multi_model_comparison_workflow(
#     client,
#     model_ids=["model1", "model2", "model3"],
#     layer_filter=["layer1", "layer2", "layer3"]
# )

## 3. Workflow: Real-Time Monitoring Dashboard

Set up real-time monitoring with custom alerting.

In [ ]:
class AnomalyMonitor:
    """
    Real-time anomaly monitoring system with custom alerting.
    """
    
    def __init__(self, client, alert_threshold=0.8):
        self.client = client
        self.alert_threshold = alert_threshold
        self.alert_count = 0
        self.start_time = None
    
    async def start(self):
        """Start monitoring"""
        await self.client.subscribe_anomalies()
        self.start_time = datetime.now()
        print(f"🚨 Anomaly Monitor Started")
        print(f"   Alert Threshold: {self.alert_threshold}")
        print(f"   Started at: {self.start_time.strftime('%H:%M:%S')}")
    
    async def stop(self):
        """Stop monitoring"""
        await self.client.unsubscribe_anomalies()
        elapsed = (datetime.now() - self.start_time).total_seconds()
        print(f"\n🛑 Monitoring Stopped")
        print(f"   Duration: {elapsed:.1f} seconds")
        print(f"   Alerts Triggered: {self.alert_count}")
    
    async def monitor(self, duration=60):
        """
        Monitor for specified duration (in seconds).
        Returns summary of alerts.
        """
        await self.start()
        
        alerts = []
        start_time = datetime.now()
        
        try:
            async for update in self.client.stream_updates(message_types=["anomaly_update"]):
                # Check if alert threshold exceeded
                if update.score >= self.alert_threshold:
                    alert = {
                        "timestamp": update.timestamp,
                        "severity": update.severity,
                        "score": update.score,
                        "description": update.description,
                        "layer": update.layer
                    }
                    alerts.append(alert)
                    self.alert_count += 1
                    
                    # Print alert
                    print(f"\n⚠️ ALERT #{self.alert_count}")
                    print(f"   Severity: {alert['severity'].upper()}")
                    print(f"   Score: {alert['score']:.3f}")
                    print(f"   Description: {alert['description']}")
                
                # Check if duration exceeded
                elapsed = (datetime.now() - start_time).total_seconds()
                if elapsed >= duration:
                    break
                    
        except asyncio.CancelledError:
            pass
        
        await self.stop()
        
        return {
            "duration": duration,
            "total_alerts": len(alerts),
            "alerts": alerts
        }

# Example usage:
# monitor = AnomalyMonitor(client, alert_threshold=0.7)
# summary = await monitor.monitor(duration=30)
# print(f"\nMonitoring Summary: {summary['total_alerts']} alerts in {summary['duration']}s")

## 4. Workflow: Automated Data Collection

Automatically collect and archive data at regular intervals.

In [ ]:
async def automated_collection_workflow(client, interval_seconds=60, num_collections=5):
    """
    Automatically collect data at regular intervals.
    
    Args:
        client: NeuralHeatmapClient instance
        interval_seconds: Time between collections
        num_collections: Number of collections to perform
    """
    archive_dir = Path("./data_archive")
    archive_dir.mkdir(exist_ok=True)
    
    print(f"📦 Automated Data Collection")
    print("=" * 50)
    print(f"Interval: {interval_seconds}s")
    print(f"Collections: {num_collections}")
    print(f"Archive: {archive_dir.absolute()}")
    print()
    
    collection_summary = []
    
    for i in range(num_collections):
        timestamp = datetime.now()
        timestamp_str = timestamp.strftime("%Y%m%d_%H%M%S")
        collection_dir = archive_dir / f"collection_{timestamp_str}"
        collection_dir.mkdir(exist_ok=True)
        
        print(f"\n📥 Collection {i+1}/{num_collections} [{timestamp.strftime('%H:%M:%S')}]")
        
        try:
            # Collect correlation matrix
            corr_df = await client.get_correlation_matrix(as_dataframe=True)
            corr_df.to_csv(collection_dir / "correlation.csv")
            print(f"  ✓ Correlation matrix")
            
            # Collect temporal patterns
            patterns = await client.get_temporal_patterns()
            with open(collection_dir / "temporal_patterns.json", "w") as f:
                json.dump([{
                    "type": p.pattern_type,
                    "confidence": p.confidence,
                    "frequency": p.frequency,
                    "metadata": p.metadata
                } for p in patterns], f)
            print(f"  ✓ Temporal patterns ({len(patterns)} found)")
            
            # Collect anomalies
            anomalies = await client.get_anomalies()
            with open(collection_dir / "anomalies.json", "w") as f:
                json.dump([{
                    "type": a.anomaly_type,
                    "severity": a.severity,
                    "score": a.score,
                    "description": a.description
                } for a in anomalies], f)
            print(f"  ✓ Anomalies ({len(anomalies)} found)")
            
            # Record summary
            collection_summary.append({
                "timestamp": timestamp.isoformat(),
                "directory": str(collection_dir),
                "patterns": len(patterns),
                "anomalies": len(anomalies)
            })
            
            print(f"  ✓ Saved to {collection_dir.name}")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
            collection_summary.append({
                "timestamp": timestamp.isoformat(),
                "error": str(e)
            })
        
        # Wait for next interval (unless it's the last collection)
        if i < num_collections - 1:
            print(f"  ⏳ Waiting {interval_seconds}s...")
            await asyncio.sleep(interval_seconds)
    
    # Save collection summary
    with open(archive_dir / "collection_summary.json", "w") as f:
        json.dump(collection_summary, f, indent=2)
    
    print("\n" + "=" * 50)
    print("✅ Data Collection Complete")
    print(f"   Collections: {len(collection_summary)}")
    print(f"   Archive: {archive_dir.absolute()}")
    
    return collection_summary

# Example usage:
# summary = await automated_collection_workflow(client, interval_seconds=30, num_collections=3)

## 5. Workflow: Custom Theme and Layout

Apply custom visualization themes and layouts.

In [ ]:
async def customization_workflow(client):
    """
    Apply custom themes and layouts to the visualization.
    """
    print("🎨 Customization Workflow")
    print("=" * 50)
    
    # Apply themes
    themes = [
        VisualizationTheme.THERMAL,
        VisualizationTheme.PLASMA,
        VisualizationTheme.HOLOGRAPHIC,
        VisualizationTheme.CONTOUR
    ]
    
    for theme in themes:
        print(f"\n🎭 Applying theme: {theme.value}")
        success = await client.set_theme(theme)
        if success:
            print(f"   ✓ Theme applied")
        else:
            print(f"   ✗ Failed to apply theme")
        await asyncio.sleep(1)  # Brief pause to see the change
    
    # Apply custom layout
    print("\n📐 Applying custom layout...")
    
    custom_layout = {
        "panel_positions": {
            "heatmap": {"x": 0, "y": 0},
            "correlation": {"x": 400, "y": 0},
            "temporal": {"x": 0, "y": 300},
            "anomalies": {"x": 400, "y": 300}
        },
        "panel_sizes": {
            "heatmap": {"width": 350, "height": 250},
            "correlation": {"width": 350, "height": 250},
            "temporal": {"width": 350, "height": 200},
            "anomalies": {"width": 350, "height": 200}
        },
        "visibility": {
            "heatmap": True,
            "correlation": True,
            "temporal": True,
            "anomalies": True
        },
        "z_order": ["heatmap", "correlation", "temporal", "anomalies"]
    }
    
    success = await client.set_layout(custom_layout)
    if success:
        print("   ✓ Custom layout applied")
    else:
        print("   ✗ Failed to apply layout")
    
    print("\n" + "=" * 50)
    print("✅ Customization Complete")

# Uncomment to run:
# await customization_workflow(client)

## 6. Workflow: Export for TensorBoard

Prepare data for visualization in TensorBoard.

In [ ]:
async def tensorboard_export_workflow(client, output_dir="./tensorboard_logs"):
    """
    Export data in TensorBoard format.
    """
    print("📊 TensorBoard Export Workflow")
    print("=" * 50)
    
    log_dir = await client.export_tensorboard(
        output_dir=output_dir,
        run_name=f"neural_heatmap_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    )
    
    print(f"\n✅ TensorBoard logs exported to:")
    print(f"   {log_dir}")
    print(f"\nTo view in TensorBoard:")
    print(f"   tensorboard --logdir={Path(log_dir).parent}")
    
    return log_dir

# Uncomment to run:
# log_dir = await tensorboard_export_workflow(client)

## 7. Workflow: Batch Export

Export all available data in multiple formats.

In [ ]:
async def batch_export_workflow(client, output_dir="./batch_export"):
    """
    Export all data in multiple formats.
    """
    print("📦 Batch Export Workflow")
    print("=" * 50)
    
    formats = [ExportFormat.CSV, ExportFormat.JSON]
    
    exported = await client.export_all(
        output_dir=output_dir,
        formats=formats
    )
    
    print(f"\n✅ Exported {len(exported)} files:")
    for data_type, filepath in exported.items():
        print(f"   {data_type}: {filepath}")
    
    return exported

# Uncomment to run:
# exported = await batch_export_workflow(client)

## 8. Cleanup

In [ ]:
async def cleanup():
    await client.disconnect()
    print("✅ Disconnected from server")

await cleanup()

## Summary

In this notebook, you learned:
- How to combine multiple API operations into workflows
- Creating automated data collection pipelines
- Building real-time monitoring systems
- Applying custom themes and layouts
- Exporting data for external tools like TensorBoard

## Next Steps
- Create your own custom workflows
- Integrate with your existing analysis pipelines
- Build automated alerting systems
- Combine with machine learning models